<a href="https://colab.research.google.com/github/GavinHacker/recsys_model/blob/master/2_process_userproex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 数据处理Part 2
* 处理用户的属性，通过用户的rat, 构造统计特征，max，min, avg, medium

#### 需要每次构建模型都执行

In [0]:
#!python --version

In [0]:
import pandas as pd
import pymysql
import pymysql.cursors
from functools import reduce
import numpy as np
import pandas as pd
import uuid
import datetime

In [0]:
def get_connection():
    return pymysql.connect(host='rm-2zeqqm6994abi7b6dqo.mysql.rds.aliyuncs.com',
                               user='noone',
                               password='Huawei12#$',
                               db='recsys',
                               port=3306,
                               charset ='utf8',
                               use_unicode=True)

## 插入和更新一条用户统计信息

In [0]:
def insert_or_update_one_userproex(userid, rmax, rmin, ravg, rmedium, rcount, rsum):
    print('insert_or_update_one_userproex start')
    is_insert = True
    connection = get_connection()
    try:
        sql = 'select count(*) from userproex where userid=\'%s\'' % userid
        with connection.cursor() as cursor:
            cursor.execute(sql)
        connection.commit()
        res_cnt = cursor.fetchall()[0][0]
        if res_cnt > 0:
            is_insert = False
    except Exception as e:
        print('query for exist info error'+str(e))
        
    if is_insert:
        sql = 'insert into userproex(userid, rmax, rmin, ravg, rcount, rsum, rmedian) values(\'%s\', %s, %s, %s, %s, %s, %s)' % (userid, rmax, rmin, ravg, rcount, rsum, rmedium)
    else:
        sql = 'update userproex set rmax=%s, rmin=%s, ravg=%s, rmedian=%s, rcount=%s, rsum=%s where userid=\'%s\'' % (rmax, rmin, ravg, rmedium, rcount, rsum, userid)
    
    try:
        with connection.cursor() as cursor:
            cursor.execute(sql)
        connection.commit()
        print(('insert userid:' if is_insert else 'update userid:') + str(userid) + ' success.')
    except Exception as e:
        print(e)
        connection.close()
    connection.close()
  
# 测试代码
# insert_or_update_one_userproex('99', 9.1234567, 3.9, 5, 7, 9, 1.1111111)

## 更新一条用户评分中位数

In [0]:
def insert_or_update_one_userproex_4_median(userid, rmax, rmin, ravg, rmedian, rcount, rsum, connection):
    print('insert_or_update_one_userproex_4_median start')
    is_insert = True
    try:
        sql = 'select count(*) from userproex_new where userid=\'%s\'' % userid
        with connection.cursor() as cursor:
            cursor.execute(sql)
        res_cnt = cursor.fetchall()[0][0]
        if res_cnt > 0:
            is_insert = False
    except Exception as e:
        print('query for exist info error'+str(e))
        
    if is_insert:
        print('异常数据'+str(userid))
        return
    else:
        sql = 'update userproex_new set rmedian=%s where userid=\'%s\'' % (rmedian, userid)
    
    try:
        with connection.cursor() as cursor:
            cursor.execute(sql)
        print(('insert userid:' if is_insert else 'update userid:') + str(userid) + ' success.')
    except Exception as e:
        print(e)
        connection.close()

## 针对每一个用户，处理统计数据其中包含max,min...等

In [0]:
def process_per_user(userid, insert_or_update_func):
    connection = get_connection()
    
    rmedian = 0
    rmax = 0
    rmin = 0
    ravg = 0
    rcount = 0
    rsum = 0
    
    sql = 'select rating from comment_new where userid=\'%s\'' % userid
    try:
        with connection.cursor() as cursor:
            count = cursor.execute(sql)
        rlist = cursor.fetchall()
        rmedian = np.median(np.array(list(map(lambda x:x[0], rlist))).astype(float))
    except Exception as e:
        print(e)
        connection.close()
    
    sql = 'select max(rating) as rmax, min(rating) as rmin, avg(rating) as ravg, count(rating) as rcount, sum(rating) as rsum from comment_new where userid=\'%s\'' % userid
    try:
        count = 0
        with connection.cursor() as cursor:
            count = cursor.execute(sql)
        if count > 0:
            r = cursor.fetchone()
            rmax, rmin, ravg, rcount, rsum = r[0], r[1], r[2], r[3], r[4]
    except Exception as e:
        print(e)
        connection.close()
    insert_or_update_func(userid=userid, rmax=rmax, rmin=rmin, ravg=ravg, rmedium=rmedian, rcount=rcount, rsum=rsum)
    connection.close()

## 全量处理数据，只处理median，由于max,min等都有内置函数，所以直接sql处理

In [0]:
def process_per_user_median(userid, insert_or_update_func, connection):
    rmedian = 0
    rmax = 0
    rmin = 0
    ravg = 0
    rcount = 0
    rsum = 0
    sql = 'select rating from comment_new where userid=\'%s\'' % userid
    try:
        with connection.cursor() as cursor:
            count = cursor.execute(sql)
        rlist = cursor.fetchall()
        rmedian = np.median(np.array(list(map(lambda x:x[0], rlist))).astype(float))
    except Exception as e:
        print(e)
        connection.close()
    
    insert_or_update_func(userid=userid, rmax=rmax, rmin=rmin, ravg=ravg, rmedian=rmedian, rcount=rcount, rsum=rsum, connection=connection)

## 全量处理数据

* 1.执行sql，处理除中位数的其他统计值

    -- delete from userproex_new
    -- INSERT INTO userproex_new (userid, rmax, rmin, ravg, rcount, rsum) (select userid, max(rating) as rmax, min(rating) as rmin, avg(rating) as ravg, count(rating) as rcount, sum(rating) as rsum from comment_new group by userid having userid is not null)

* 2.执行中位数更新

In [0]:
def get_user_list_in_comment(is_all_or_sample, process_func, insert_or_update_func):
    connection = get_connection()
    if is_all_or_sample:
        sql = 'select DISTINCT userid from comment_new where userid is not null and userid in (select userid from userproex_new where rmedian=0)'
    else:
        sql = 'select DISTINCT userid from comment_new where userid is not null and userid in (select userid from userproex_new where rmedian=0) limit 0,10'
        
    try:
        with connection.cursor() as cursor:
            cursor.execute(sql)
        connection.commit()
        process_size = 0
        while True:
            r = cursor.fetchone()
            if r == None:
                break
            process_func(r[0], insert_or_update_func, connection)
            process_size = process_size + 1
            if process_size >= 1000:
                connection.commit()
                process_size = 0
        connection.commit()
    except Exception as e:
        print(e)
        connection.close()
    connection.close()

In [0]:
print(datetime.datetime.now())
start_time = datetime.datetime.now()

2018-12-19 15:58:07.804579


## 执行主函数

In [0]:
get_user_list_in_comment(True, process_per_user_median, insert_or_update_one_userproex_4_median)

insert_or_update_one_userproex_4_median start
update userid:ff9a0434038d5ad817c7a2b603eae73e success.
insert_or_update_one_userproex_4_median start
update userid:ff44570aca8241914870afbc310cdb85 success.
insert_or_update_one_userproex_4_median start
update userid:ff5bb67db636a8cf08cac6617f1954ca success.
insert_or_update_one_userproex_4_median start
update userid:feb77f1c2ec488cfb38acb7c602573f8 success.
insert_or_update_one_userproex_4_median start
update userid:ffa23ca940c927a962d02c57b0eb9700 success.
insert_or_update_one_userproex_4_median start
update userid:fec6bb6d24d9e5eb405ffe22420786e6 success.
insert_or_update_one_userproex_4_median start
update userid:fed9f34b314fb2aa0d7c59593fa35241 success.
insert_or_update_one_userproex_4_median start
update userid:ffbd23a0248e1ee2ae890fafbc4240b6 success.
insert_or_update_one_userproex_4_median start
update userid:fed792c007ce53a4c86d175139bd08cd success.
insert_or_update_one_userproex_4_median start
update userid:fe792c95601128531e2c53

In [0]:
print(datetime.datetime.now())
end_time = datetime.datetime.now()
print(end_time - start_time)

2018-12-19 11:10:31.568195
0:00:17.952835


## 增量处理数据（这部分在实际的工程代码中，替换为使用数据库的mqlog表，把这张表当做消息队列来使用，和log思路一致，针对user这部分,  mqlog中的消息类型为‘u’）

* 根据支持，传入user list

* 根据user list, 更新max，min, avg, sum, count， median等值

In [0]:
def get_user_list_in_log(is_all_or_sample, process_func, insert_or_update_func):
    connection = get_connection()
    if is_all_or_sample:
        sql = 'select DISTINCT userid from comment_new_log where userid is not null and active=1'
    else:
        sql = 'select DISTINCT userid from comment_new_log where userid is not null and active=1 limit 0,10'
        
    try:
        with connection.cursor() as cursor:
            cursor.execute(sql)
        connection.commit()
        while True:
            r = cursor.fetchone()
            if r == None:
                break
            process_func(r[0], insert_or_update_func)
            with connection.cursor() as cursor:
                update_sql = 'update comment_new_log set active=0 where userid=\'%s\'' % r[0]
                cursor.execute(update_sql)
            connection.commit()
    except Exception as e:
        print(e)
        connection.close()
    connection.close()

In [0]:
#get_user_list_in_log(False, process_per_user, insert_or_update_one_userproex)